In [50]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("lin_reg").getOrCreate()

In [4]:
from pyspark.ml.regression import LinearRegression

In [5]:
training = spark.read.format('libsvm').load('sample_linear_regression_data.txt')

In [6]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [7]:
lr = LinearRegression(featuresCol='features', labelCol='label', 
                      predictionCol='prediction')

In [8]:
lr_model = lr.fit(training)

In [9]:
lr_model.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [10]:
lr_model.intercept

0.14228558260358093

In [11]:
training_summary = lr_model.summary

In [12]:
training_summary.r2

0.027839179518600154

In [13]:
training_summary.meanAbsoluteError

8.145215527783876

In [14]:
all_data = spark.read.format("libsvm").load("sample_linear_regression_data.txt")

In [15]:
split_object = all_data.randomSplit([0.7, 0.3])

In [18]:
split_object

[DataFrame[label: double, features: vector],
 DataFrame[label: double, features: vector]]

In [19]:
split_object[0].show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
|-28.571478869743427|(10,[0,1,2,3,4,5,...|
|-28.046018037776633|(10,[0,1,2,3,4,5,...|
|-26.805483428483072|(10,[0,1,2,3,4,5,...|
| -23.51088409032297|(10,[0,1,2,3,4,5,...|
|-23.487440120936512|(10,[0,1,2,3,4,5,...|
|-22.949825936196074|(10,[0,1,2,3,4,5,...|
|-22.837460416919342|(10,[0,1,2,3,4,5,...|
|-21.432387764165806|(10,[0,1,2,3,4,5,...|
|-20.212077258958672|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-19.884560774273424|(10,[0,1,2,3,4,5,...|
|-19.872991038068406|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -19.66731861537172|(10,[0,1,2,3,4,5,...|
|-19.402336030214553|(10,[0,1,2,3,4,5,...|
|-18.845922472898582|(10,[0,1,2,3,4,5,...|
| -18.27521356600463|(10,[0,1,2,3,4,5,...|
|-17.803626188664516|(10,[0,1,2,3,4,5,...|
|-17.494200356883344|(10,[0,1,2,3,4,5,...|
|-17.026492264209548|(10,[0,1,2,3,4,5,...|
+----------

In [20]:
train_data, test_data = all_data.randomSplit([0.7,0.3])

In [21]:
train_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                337|
|   mean|0.20964005282862394|
| stddev| 10.648345034290092|
|    min|-28.571478869743427|
|    max|  27.78383192005107|
+-------+-------------------+



In [22]:
test_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                164|
|   mean|  0.353979281197826|
| stddev|  9.634087768520722|
|    min|-22.837460416919342|
|    max| 27.111027963108548|
+-------+-------------------+



In [23]:
correct_model = lr.fit(train_data)

In [24]:
test_results = correct_model.evaluate(test_data)

In [25]:
test_results.rootMeanSquaredError

9.63723015576627

In [26]:
unlabeled_data = test_data.select('features')

In [27]:
predictions = correct_model.transform(unlabeled_data)

In [28]:
predictions.show()

+--------------------+--------------------+
|            features|          prediction|
+--------------------+--------------------+
|(10,[0,1,2,3,4,5,...|  -4.472663522127767|
|(10,[0,1,2,3,4,5,...|  1.4541381393228008|
|(10,[0,1,2,3,4,5,...|-0.07892751217990854|
|(10,[0,1,2,3,4,5,...| -1.2490663451693922|
|(10,[0,1,2,3,4,5,...|  -2.171793344578192|
|(10,[0,1,2,3,4,5,...|  1.0650544218969564|
|(10,[0,1,2,3,4,5,...| -0.5052610882602364|
|(10,[0,1,2,3,4,5,...| -0.3373421934819487|
|(10,[0,1,2,3,4,5,...| -0.8557822149372614|
|(10,[0,1,2,3,4,5,...| -1.1785739675345974|
|(10,[0,1,2,3,4,5,...|   3.172267007599983|
|(10,[0,1,2,3,4,5,...|   1.074274471312284|
|(10,[0,1,2,3,4,5,...|  3.0346503774365683|
|(10,[0,1,2,3,4,5,...| -1.2684669496407583|
|(10,[0,1,2,3,4,5,...| 0.07377802366127892|
|(10,[0,1,2,3,4,5,...|  2.4060655937832087|
|(10,[0,1,2,3,4,5,...| -0.5682003253390328|
|(10,[0,1,2,3,4,5,...| -1.5145395701060997|
|(10,[0,1,2,3,4,5,...| -1.3282371103345838|
|(10,[0,1,2,3,4,5,...|-0.0045994

# Linear Regression Example Code Along

In [51]:
data = spark.read.csv('Ecommerce_Customers.csv', inferSchema=True, header=True)

In [31]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [35]:
for item in data.head(1)[0]:
    print (item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


In [52]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [37]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [53]:
assembler = VectorAssembler(inputCols=['Avg Session Length', 'Time on App', 'Time on Website', 'Length of Membership'],
                            outputCol='features')

In [54]:
output = assembler.transform(data)

In [56]:
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [57]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [58]:
final_data = output.select('features', 'Yearly Amount Spent')

In [59]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [60]:
training_data, test_data = final_data.randomSplit([0.7,0.3])

In [61]:
training_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                342|
|   mean| 500.09929145823884|
| stddev|  79.20156684908798|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [65]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                158|
|   mean|  497.6143129783405|
| stddev|  79.78491073272022|
|    min| 308.52774655803336|
|    max|  725.5848140556806|
+-------+-------------------+



In [67]:
lr2= LinearRegression(labelCol='Yearly Amount Spent')

In [68]:
lr_model = lr2.fit(training_data)

In [69]:
test_results = lr_model.evaluate(test_data)

In [70]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
| 11.806454375096564|
|-10.974069884965616|
| 0.9970209902064084|
|-12.525283310524571|
| 1.8628246044013395|
| 3.6921875395382244|
| -6.357362101307331|
|-1.6644731779994117|
|-0.5160781960563554|
|-26.041403039050806|
| 2.2344183074868056|
|  8.297974306848062|
|-1.8180632195155795|
|-10.115317745622292|
| -5.096029618235377|
| 12.701706014097738|
|-13.238118742471215|
|  5.995987133913445|
|-17.762317373548058|
|  5.831049425745277|
+-------------------+
only showing top 20 rows



In [71]:
test_results.rootMeanSquaredError

9.572462279233076

In [72]:
test_results.r2

0.9855135078658035

In [73]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [74]:
unlabeled_data = test_data.select('features')

In [76]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[29.5324289670579...|
|[30.3931845423455...|
|[30.5743636841713...|
|[31.0662181616375...|
|[31.3895854806643...|
|[31.4459724827577...|
|[31.4474464941278...|
|[31.5761319713222...|
|[31.6610498227460...|
|[31.6739155032749...|
|[31.7366356860502...|
|[31.8209982016720...|
|[31.8530748017465...|
|[31.8648325480987...|
|[31.8745516945853...|
|[31.9262720263601...|
|[31.9365486184489...|
|[31.9480174211613...|
|[31.9563005605233...|
|[31.9764800614612...|
+--------------------+
only showing top 20 rows



In [77]:
predictions = lr_model.transform(unlabeled_data)

In [78]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[29.5324289670579...| 396.8338966975309|
|[30.3931845423455...| 330.9029396881592|
|[30.5743636841713...|441.06739276785925|
|[31.0662181616375...| 461.4585765181989|
|[31.3895854806643...|408.20678645558155|
|[31.4459724827577...|481.18477739559034|
|[31.4474464941278...|424.96010419653135|
|[31.5761319713222...| 542.8910571673277|
|[31.6610498227460...| 416.8744317759572|
|[31.6739155032749...|  501.766470948932|
|[31.7366356860502...| 494.6990279480451|
|[31.8209982016720...| 416.3773067063653|
|[31.8530748017465...|461.10318668186756|
|[31.8648325480987...|450.00659822243597|
|[31.8745516945853...| 397.3812738645029|
|[31.9262720263601...|379.50322743022866|
|[31.9365486184489...| 440.4375036377994|
|[31.9480174211613...|455.92488975898436|
|[31.9563005605233...| 564.8882491207469|
|[31.9764800614612...| 324.7633966083549|
+--------------------+------------